In [ ]:
# Comprehensive Exploratory Data Analysis - Parkinson's Telemonitoring Dataset

import os
import ssl
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import warnings

warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# ---------------------------------------------------------
# Configuration
# ---------------------------------------------------------
DATA_URL = "https://archive.ics.uci.edu/static/public/189/parkinsons+telemonitoring.zip"
TARGET_DIR = "../data/raw/"
ZIP_PATH = os.path.join(TARGET_DIR, "parkinsons_telemonitoring.zip")

# ---------------------------------------------------------
# Ensure folder exists
# ---------------------------------------------------------
os.makedirs(TARGET_DIR, exist_ok=True)

# ---------------------------------------------------------
# SSL context (handles macOS CA issues)
# ---------------------------------------------------------
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# ---------------------------------------------------------
# Download
# ---------------------------------------------------------
def download():
    print(f"Downloading Land Mines dataset from UCI...")
    try:
        with urllib.request.urlopen(DATA_URL, context=ssl_context) as response:
            data = response.read()
            with open(ZIP_PATH, "wb") as f:
                f.write(data)
    except Exception as e:
        print(f"Download failed: {e}")
        raise

    size_kb = os.path.getsize(ZIP_PATH) / 1024
    print(f"Saved to {ZIP_PATH} ({size_kb:.1f} KB)")


# ---------------------------------------------------------
# Unzip the archive
# ---------------------------------------------------------
def unzip():
    import zipfile
    with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(TARGET_DIR)
    print(f"Extracted contents into {TARGET_DIR}")


# ---------------------------------------------------------
# Main execution
# ---------------------------------------------------------
if __name__ == "__main__":
    download()
    unzip()
    print("Done.")


In [ ]:
dataset = pd.read_csv("../data/raw/parkinsons_updrs.data")
print(dataset.head())

In [ ]:
# ============================================================================
# 1. BASIC DATASET INFORMATION
# ============================================================================

print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nDataset shape: {dataset.shape}")
print(f"Number of samples: {dataset.shape[0]}")
print(f"Number of features: {dataset.shape[1]}")

print("\n" + "=" * 80)
print("COLUMN INFORMATION")
print("=" * 80)
print(dataset.info())

print("\n" + "=" * 80)
print("MISSING VALUES")
print("=" * 80)
missing = dataset.isnull().sum()
if missing.any():
    print(missing[missing > 0])
else:
    print("No missing values found!")

print("\n" + "=" * 80)
print("DATA TYPES")
print("=" * 80)
print(dataset.dtypes.value_counts())

print("\n" + "=" * 80)
print("BASIC STATISTICS")
print("=" * 80)
print(dataset.describe().T)

In [ ]:
# ============================================================================
# 2. TARGET VARIABLE ANALYSIS
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Motor UPDRS distribution
axes[0, 0].hist(dataset['motor_UPDRS'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Motor UPDRS Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Motor UPDRS Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(dataset['motor_UPDRS'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {dataset["motor_UPDRS"].mean():.2f}')
axes[0, 0].legend()

# Total UPDRS distribution
axes[0, 1].hist(dataset['total_UPDRS'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Total UPDRS Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Total UPDRS Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(dataset['total_UPDRS'].mean(), color='red', linestyle='--',
                   label=f'Mean: {dataset["total_UPDRS"].mean():.2f}')
axes[0, 1].legend()

# Motor UPDRS Q-Q plot
stats.probplot(dataset['motor_UPDRS'], dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Motor UPDRS Q-Q Plot', fontsize=14, fontweight='bold')

# Total UPDRS Q-Q plot
stats.probplot(dataset['total_UPDRS'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Total UPDRS Q-Q Plot', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("TARGET VARIABLE STATISTICS")
print("=" * 80)
print("\nMotor UPDRS:")
print(f"  Mean: {dataset['motor_UPDRS'].mean():.2f}")
print(f"  Std:  {dataset['motor_UPDRS'].std():.2f}")
print(f"  Min:  {dataset['motor_UPDRS'].min():.2f}")
print(f"  Max:  {dataset['motor_UPDRS'].max():.2f}")
print(f"  Skewness: {dataset['motor_UPDRS'].skew():.2f}")
print(f"  Kurtosis: {dataset['motor_UPDRS'].kurtosis():.2f}")

print("\nTotal UPDRS:")
print(f"  Mean: {dataset['total_UPDRS'].mean():.2f}")
print(f"  Std:  {dataset['total_UPDRS'].std():.2f}")
print(f"  Min:  {dataset['total_UPDRS'].min():.2f}")
print(f"  Max:  {dataset['total_UPDRS'].max():.2f}")
print(f"  Skewness: {dataset['total_UPDRS'].skew():.2f}")
print(f"  Kurtosis: {dataset['total_UPDRS'].kurtosis():.2f}")

In [ ]:
# ============================================================================
# 3. FEATURE DISTRIBUTIONS
# ============================================================================

# Separate features from targets
feature_cols = [col for col in dataset.columns if col not in ['subject#', 'motor_UPDRS', 'total_UPDRS']]
features = dataset[feature_cols]

# Plot all feature distributions
n_features = len(feature_cols)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 3))
axes = axes.flatten()

for idx, col in enumerate(feature_cols):
    axes[idx].hist(features[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(col, fontsize=10, fontweight='bold')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

# Hide empty subplots
for idx in range(n_features, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Feature Distributions', fontsize=16, fontweight='bold', y=1.001)
plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("FEATURE STATISTICS SUMMARY")
print("=" * 80)
print(features.describe().T[['mean', 'std', 'min', 'max']])

In [ ]:
# ============================================================================
# 4. CORRELATION ANALYSIS
# ============================================================================

# Compute correlation matrix
correlation_matrix = dataset[feature_cols + ['motor_UPDRS', 'total_UPDRS']].corr()

# Plot full correlation heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Top correlations with target variables
print("\n" + "=" * 80)
print("TOP CORRELATIONS WITH MOTOR_UPDRS")
print("=" * 80)
motor_corr = correlation_matrix['motor_UPDRS'].drop(['motor_UPDRS', 'total_UPDRS']).sort_values(ascending=False)
print(motor_corr.head(10))

print("\n" + "=" * 80)
print("TOP CORRELATIONS WITH TOTAL_UPDRS")
print("=" * 80)
total_corr = correlation_matrix['total_UPDRS'].drop(['motor_UPDRS', 'total_UPDRS']).sort_values(ascending=False)
print(total_corr.head(10))

# Visualize top correlations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Motor UPDRS correlations
motor_corr.head(10).plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Top 10 Features Correlated with Motor UPDRS', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Correlation Coefficient')
axes[0].grid(True, alpha=0.3)

# Total UPDRS correlations
total_corr.head(10).plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Top 10 Features Correlated with Total UPDRS', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Correlation Coefficient')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 5. PRINCIPAL COMPONENT ANALYSIS (PCA)
# ============================================================================

print("=" * 80)
print("PRINCIPAL COMPONENT ANALYSIS")
print("=" * 80)

# Standardize features for PCA
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Fit PCA
pca = PCA()
pca_features = pca.fit_transform(features_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"\nTotal features: {features_scaled.shape[1]}")
print(f"Total components: {len(explained_variance)}")

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nComponents needed for 95% variance: {n_components_95}")
print(f"Components needed for 90% variance: {np.argmax(cumulative_variance >= 0.90) + 1}")

# Plot explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance * 100, alpha=0.7, color='steelblue')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance (%)')
axes[0].set_title('Scree Plot - Explained Variance by Component', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
axes[0].legend()

# Cumulative explained variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance * 100, 
             marker='o', linewidth=2, color='darkgreen')
axes[1].axhline(y=95, color='red', linestyle='--', alpha=0.7, label='95% variance')
axes[1].axhline(y=90, color='orange', linestyle='--', alpha=0.7, label='90% variance')
axes[1].axvline(x=n_components_95, color='red', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance (%)')
axes[1].set_title('Cumulative Explained Variance', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

# Print variance per component
print("\n" + "=" * 80)
print("EXPLAINED VARIANCE PER COMPONENT (Top 10)")
print("=" * 80)
for i in range(min(10, len(explained_variance))):
    print(f"PC{i+1:2d}: {explained_variance[i]*100:6.2f}% | Cumulative: {cumulative_variance[i]*100:6.2f}%")

In [ ]:
# ============================================================================
# 6. PCA COMPONENT LOADINGS - Feature Importance
# ============================================================================

# Get component loadings for first few PCs
n_top_pcs = 5
loadings = pca.components_[:n_top_pcs, :]

# Create dataframe of loadings
loadings_df = pd.DataFrame(
    loadings.T,
    columns=[f'PC{i+1}' for i in range(n_top_pcs)],
    index=feature_cols
)

# Plot loadings for first 3 PCs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i in range(3):
    pc_loadings = loadings_df[f'PC{i+1}'].sort_values(ascending=False)
    
    axes[i].barh(range(len(pc_loadings)), pc_loadings.values, color='steelblue', alpha=0.7)
    axes[i].set_yticks(range(len(pc_loadings)))
    axes[i].set_yticklabels(pc_loadings.index, fontsize=8)
    axes[i].set_xlabel('Loading Value')
    axes[i].set_title(f'PC{i+1} Feature Loadings\n({explained_variance[i]*100:.1f}% variance)', 
                     fontsize=11, fontweight='bold')
    axes[i].axvline(x=0, color='red', linestyle='--', alpha=0.5)
    axes[i].grid(True, alpha=0.3, axis='x')
    axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("TOP FEATURES BY ABSOLUTE LOADING - PC1")
print("=" * 80)
print(loadings_df['PC1'].abs().sort_values(ascending=False).head(10))

In [ ]:
# ============================================================================
# 7. PCA VISUALIZATION - 2D and 3D Projections
# ============================================================================

# Create PCA projections colored by target variables
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# PC1 vs PC2 - Motor UPDRS
scatter1 = axes[0, 0].scatter(pca_features[:, 0], pca_features[:, 1], 
                              c=dataset['motor_UPDRS'], cmap='viridis', alpha=0.6, s=20)
axes[0, 0].set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
axes[0, 0].set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
axes[0, 0].set_title('PCA Projection - Colored by Motor UPDRS', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0, 0], label='Motor UPDRS')
axes[0, 0].grid(True, alpha=0.3)

# PC1 vs PC2 - Total UPDRS
scatter2 = axes[0, 1].scatter(pca_features[:, 0], pca_features[:, 1], 
                              c=dataset['total_UPDRS'], cmap='plasma', alpha=0.6, s=20)
axes[0, 1].set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
axes[0, 1].set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
axes[0, 1].set_title('PCA Projection - Colored by Total UPDRS', fontweight='bold')
plt.colorbar(scatter2, ax=axes[0, 1], label='Total UPDRS')
axes[0, 1].grid(True, alpha=0.3)

# PC2 vs PC3 - Motor UPDRS
scatter3 = axes[1, 0].scatter(pca_features[:, 1], pca_features[:, 2], 
                              c=dataset['motor_UPDRS'], cmap='viridis', alpha=0.6, s=20)
axes[1, 0].set_xlabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
axes[1, 0].set_ylabel(f'PC3 ({explained_variance[2]*100:.1f}%)')
axes[1, 0].set_title('PC2 vs PC3 - Colored by Motor UPDRS', fontweight='bold')
plt.colorbar(scatter3, ax=axes[1, 0], label='Motor UPDRS')
axes[1, 0].grid(True, alpha=0.3)

# PC2 vs PC3 - Total UPDRS
scatter4 = axes[1, 1].scatter(pca_features[:, 1], pca_features[:, 2], 
                              c=dataset['total_UPDRS'], cmap='plasma', alpha=0.6, s=20)
axes[1, 1].set_xlabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
axes[1, 1].set_ylabel(f'PC3 ({explained_variance[2]*100:.1f}%)')
axes[1, 1].set_title('PC2 vs PC3 - Colored by Total UPDRS', fontweight='bold')
plt.colorbar(scatter4, ax=axes[1, 1], label='Total UPDRS')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 8. OUTLIER DETECTION WITH ISOLATION FOREST
# ============================================================================

print("=" * 80)
print("OUTLIER DETECTION")
print("=" * 80)

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42, n_estimators=100)
outlier_labels = iso_forest.fit_predict(features_scaled)

# Count outliers
n_outliers = np.sum(outlier_labels == -1)
n_inliers = np.sum(outlier_labels == 1)

print(f"\nTotal samples: {len(outlier_labels)}")
print(f"Inliers: {n_inliers} ({n_inliers/len(outlier_labels)*100:.1f}%)")
print(f"Outliers: {n_outliers} ({n_outliers/len(outlier_labels)*100:.1f}%)")

# Visualize outliers in PCA space
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PC1 vs PC2
axes[0].scatter(pca_features[outlier_labels == 1, 0], 
                pca_features[outlier_labels == 1, 1],
                c='blue', alpha=0.6, s=20, label='Inliers')
axes[0].scatter(pca_features[outlier_labels == -1, 0], 
                pca_features[outlier_labels == -1, 1],
                c='red', alpha=0.8, s=40, marker='x', label='Outliers')
axes[0].set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
axes[0].set_title('Outlier Detection - PC1 vs PC2', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Target distribution for outliers vs inliers
motor_inliers = dataset.loc[outlier_labels == 1, 'motor_UPDRS']
motor_outliers = dataset.loc[outlier_labels == -1, 'motor_UPDRS']

axes[1].hist(motor_inliers, bins=30, alpha=0.6, label='Inliers', color='blue', edgecolor='black')
axes[1].hist(motor_outliers, bins=15, alpha=0.7, label='Outliers', color='red', edgecolor='black')
axes[1].set_xlabel('Motor UPDRS')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Motor UPDRS Distribution: Outliers vs Inliers', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics comparison
print("\n" + "=" * 80)
print("MOTOR UPDRS STATISTICS: OUTLIERS VS INLIERS")
print("=" * 80)
print(f"\nInliers - Mean: {motor_inliers.mean():.2f}, Std: {motor_inliers.std():.2f}")
print(f"Outliers - Mean: {motor_outliers.mean():.2f}, Std: {motor_outliers.std():.2f}")

In [ ]:
# ============================================================================
# 9. FEATURE PAIRWISE RELATIONSHIPS (Top Correlated Features)
# ============================================================================

# Select top features correlated with targets
top_features = motor_corr.abs().head(5).index.tolist()

# Create pairplot for selected features
plot_data = dataset[top_features + ['motor_UPDRS', 'total_UPDRS']].copy()

# Sample if dataset is too large for pairplot
if len(plot_data) > 1000:
    plot_data_sample = plot_data.sample(n=1000, random_state=42)
else:
    plot_data_sample = plot_data

print("=" * 80)
print("PAIRWISE FEATURE RELATIONSHIPS")
print("=" * 80)
print(f"\nPlotting top 5 features correlated with Motor UPDRS:")
print(top_features)

sns.pairplot(plot_data_sample, 
             diag_kind='kde',
             plot_kws={'alpha': 0.6, 's': 20},
             corner=True)
plt.suptitle('Pairwise Relationships - Top Features', y=1.001, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 10. TIME-SERIES ANALYSIS (Subject Progression)
# ============================================================================

print("=" * 80)
print("SUBJECT PROGRESSION OVER TIME")
print("=" * 80)

# Analyze temporal patterns
n_subjects = dataset['subject#'].nunique()
print(f"\nNumber of unique subjects: {n_subjects}")

# Get subjects with most recordings
subject_counts = dataset['subject#'].value_counts()
print(f"\nRecordings per subject:")
print(f"  Mean: {subject_counts.mean():.1f}")
print(f"  Median: {subject_counts.median():.1f}")
print(f"  Min: {subject_counts.min()}")
print(f"  Max: {subject_counts.max()}")

# Plot progression for a few example subjects
example_subjects = subject_counts.head(6).index

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, subject_id in enumerate(example_subjects):
    subject_data = dataset[dataset['subject#'] == subject_id].sort_values('test_time')
    
    axes[idx].plot(subject_data['test_time'], subject_data['motor_UPDRS'], 
                   marker='o', linewidth=2, label='Motor UPDRS', color='steelblue')
    axes[idx].plot(subject_data['test_time'], subject_data['total_UPDRS'], 
                   marker='s', linewidth=2, label='Total UPDRS', color='coral', alpha=0.7)
    axes[idx].set_xlabel('Test Time (days)')
    axes[idx].set_ylabel('UPDRS Score')
    axes[idx].set_title(f'Subject {subject_id} ({len(subject_data)} recordings)', 
                       fontsize=10, fontweight='bold')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Disease Progression Over Time - Sample Subjects', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 11. DEMOGRAPHIC ANALYSIS
# ============================================================================

print("=" * 80)
print("DEMOGRAPHIC ANALYSIS")
print("=" * 80)

# Age and gender distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Age distribution
axes[0].hist(dataset['age'], bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Age Distribution', fontweight='bold')
axes[0].axvline(dataset['age'].mean(), color='red', linestyle='--', 
                label=f'Mean: {dataset["age"].mean():.1f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gender distribution
gender_counts = dataset['sex'].value_counts()
axes[1].bar(['Male' if x == 0 else 'Female' for x in gender_counts.index], 
            gender_counts.values, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Count')
axes[1].set_title('Gender Distribution', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# UPDRS by gender
male_data = dataset[dataset['sex'] == 0]['motor_UPDRS']
female_data = dataset[dataset['sex'] == 1]['motor_UPDRS']

axes[2].boxplot([male_data, female_data], labels=['Male', 'Female'], patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[2].set_ylabel('Motor UPDRS Score')
axes[2].set_title('Motor UPDRS by Gender', fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nAge Statistics:")
print(f"  Mean: {dataset['age'].mean():.1f} years")
print(f"  Std:  {dataset['age'].std():.1f} years")
print(f"  Range: {dataset['age'].min():.0f} - {dataset['age'].max():.0f} years")

print(f"\nGender Distribution:")
print(f"  Male (0):   {(dataset['sex'] == 0).sum()} ({(dataset['sex'] == 0).mean()*100:.1f}%)")
print(f"  Female (1): {(dataset['sex'] == 1).sum()} ({(dataset['sex'] == 1).mean()*100:.1f}%)")

print(f"\nMotor UPDRS by Gender:")
print(f"  Male:   {male_data.mean():.2f} ± {male_data.std():.2f}")
print(f"  Female: {female_data.mean():.2f} ± {female_data.std():.2f}")

# Key Findings and Insights

## Summary of EDA

### Dataset Characteristics
- **Size**: 5,875 voice recordings from multiple subjects
- **Features**: 19 voice-related features after excluding subject ID and targets
- **Targets**: Motor UPDRS and Total UPDRS scores (continuous regression)
- **Temporal**: Multiple recordings per subject over time

### PCA Insights
- The first few principal components capture significant variance in the data
- Dimensionality reduction shows clear structure in the feature space
- PCA can be useful for visualization and potentially as input to neural networks

### Correlations
- Several voice features show moderate correlation with UPDRS scores
- Motor and Total UPDRS are highly correlated (expected)
- Feature engineering may benefit from understanding these relationships

### Outliers
- Isolation Forest detected ~5% outliers in the feature space
- Outliers show different UPDRS score distributions
- Consider robust training strategies or outlier removal

### Temporal Patterns
- Disease progression varies significantly across subjects
- Some subjects show clear upward trends in UPDRS scores
- Time-aware features or models could improve predictions

### Demographics
- Age and gender distributions are reasonably balanced
- Minor differences in UPDRS scores between demographics

## Recommendations for Neural Network

1. **Feature Scaling**: StandardScaler is essential (features have different scales)
2. **Train/Val/Test Split**: Use stratified or time-aware splitting for subjects
3. **Architecture**: Start with moderate capacity (hidden layers: 64-32-16)
4. **Regularization**: Dropout and weight decay to prevent overfitting
5. **Loss Function**: MSE for regression tasks
6. **Evaluation**: Use RMSE, MAE, and R² for interpretability
7. **Consider**: 
   - Multi-task learning for both UPDRS targets
   - Temporal models (LSTM/RNN) if leveraging time-series nature
   - Ensemble methods for robustness